## Задача 4, вариант 3.

Подключаем необходимые библиотеки:

In [1]:
import pandas
import numpy as np
import io
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
data = pandas.read_csv('C:\\Users\\79251\\Downloads\\aug_train.csv')

In [2]:
data.shape
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

Разбиваем данные на классы для построения классификатора

In [151]:
data_sel = data.loc [:, data.columns.isin(['gender', 'major_discipline','company_size', 'relevent_experience', 'education_level','training_hours', 'experience','target'])]
data_sel = data_sel.dropna()
data_sel['gender'] = np.where(data_sel['gender'] == 'Male', 0,1)
data_sel['major_discipline']=np.where((data_sel['major_discipline']=='Arts') | (data_sel['major_discipline']=='Humanities'),0,1)
data_sel['education_level'] = np.where(data_sel['education_level'] == 'Graduate', 0,1)
data_sel['relevent_experience'] = np.where(data_sel['relevent_experience'] == 'Has relevent experience', 1,0)
data_sel['experience'] = np.where(data_sel['experience'] == '>20', 0,1)
data_sel['company_size']=np.where((data_sel['company_size']=='50-99') | (data_sel['company_size']=='100-500') ,0,1)                         
Gender = data_sel.loc[:, data_sel.columns.isin(['gender'])]
X = data_sel.loc[:, data_sel.columns.isin(['major_discipline','company_size','education_level', 'relevent_experience','training_hours', 'experience','target'])]

X_train, X_test, y_train, y_test = train_test_split(X, Gender, test_size=0.3)


In [152]:
clf = SGDClassifier(loss="log", penalty="l2")
clf.fit(X_train, y_train)

C:\Users\79251\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SGDClassifier(loss='log')

In [153]:
y_pred = clf.predict(X_test)
clf.score(X_test, y_test)

0.8907180385288966

In [154]:
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.model_selection import cross_val_score
print("f1:"+str(np.average(cross_val_score(clf,X_test,y_test,cv=3, scoring="f1_micro"))))
print("accuracy:"+str(np.average(cross_val_score(clf,X_test,y_test,cv=3, scoring='accuracy'))))
print("recall:"+str(np.average(cross_val_score(clf,X_test,y_test,cv=2,scoring='recall'))))
print('precision:'+str(np.average(cross_val_score(clf,X_test,y_test,cv=2,scoring='precision'))))

C:\Users\79251\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

f1:0.8903671352873431
accuracy:0.8826658949594558
recall:0.4827136333985649
precision:0.4423076923076923


C:\Users\79251\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


accuracy:0.88

f1:0.89

precision:0.44

recall:0.48

Построим классификатор типа Случайный лест для того же классификатора.

In [156]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

X = data_sel.loc[:, data_sel.columns.isin(['major_discipline','company_size','education_level', 'relevent_experience','training_hours', 'experience','target'])]
X_train, X_test, y_train, y_test = train_test_split(X, Gender, test_size=0.3)
param_grid = { 'n_estimators': [ 200,225,250],'max_features': ['auto'],'max_depth' : list(range(1, 10)), 'criterion' :['gini']}

RFC = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv= 5, refit = True) 
RFC.fit(X_train, y_train)
Rf = RandomForestClassifier()
Rf.fit(X_train,y_train)

print("f1:"+str(np.average(cross_val_score(Rf,X_test,y_test,cv=3, scoring="f1_micro"))))
print("accuracy:"+str(np.average(cross_val_score(Rf,X_test,y_test,cv=3, scoring='accuracy'))))
print("recall:"+str(np.average(cross_val_score(Rf,X_test,y_test,cv=2,scoring='recall'))))
print('precision:'+str(np.average(cross_val_score(Rf,X_test,y_test,cv=2,scoring='precision'))))

C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y wa

f1:0.8623437808351522


C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


accuracy:0.8623456217397579


C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


recall:0.08604830421377184


C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79251\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


precision:0.13644103873988933


#### Вывод:

           Metrics                 Accuracy     f1    recall    precision
***          
    SVM:                            0.88      0.89     0.44        0.48
    Random Forest:                  0.86      0.86     0.09        0.14

Лучше оказался классификатор типа SVM (метод опорных векторов).